In [1]:
import sys
sys.path.insert(1, "C:\pdfminer-20140328")


In [2]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from cStringIO import StringIO

def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = file(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text



In [3]:
from os import path
from glob import glob 

def find_ext(dr, ext):
    return glob(path.join(dr,"*.{}".format(ext)))



In [4]:
liste = find_ext("./tazadata/","pdf")
for e in liste:
    text_file = open(str(e) + ".txt", "w")
    text_file.write(convert_pdf_to_txt(e))
    text_file.close()

In [46]:
# -*- coding: utf-8 -*-
import pandas as pd

liste = find_ext("./tazadata/","txt")

valuesTaza = []

for f in liste:
    lines = [line.rstrip('\n') for line in open(f)]
    #print str(f)[-29:]
    
    idxMalla = lines.index("Malla")
    idxItem = lines.index("Item")
    idxDefectos = lines.index("Defectos")
    idxCarac = lines.index("Características")
    idxTaza = lines.index("Tabla 5. Descripción de la taza")
    chapterIndexes = [idxMalla,idxItem,idxDefectos,idxCarac,idxTaza]
    
    #################################################
    
    #infos générales
    idxInfo = lines.index("A. DATOS DE LA MUESTRA")
    idxEndInfo = lines.index("B. ANÁLISIS FÍSICO")
    
    Tabla0Item = ["Nombre Caficultor","Municipio","Vereda","Finca","SICA","Grupo","Fecha Recpt","Fecha Analysis","No Orden"]
    Tabla0val= []
    
    decalage = 0
    
    nom = lines[idxInfo + 1]
    
    if(lines[idxInfo + 2].find("Nombre") != -1):
        decalage += 1
    
    municipio_vereda = lines[idxInfo + 2 + decalage] 
    
    
    
    '''finca_sica =  lines[idxInfo + 3]
    grupo =  lines[idxInfo + 4]
    
    fechaRec =  lines[idxInfo + 6]
    fechaAnalysis =  lines[idxInfo + 7]
    nbOrden =  lines[idxInfo + 8]'''
    
    
    # Obbligé car certain noms font 3 kilomètres et sont sur 2 lignes et c'est la manière la plus rapide (à coder)
    for i in range(idxInfo, idxEndInfo):
        l = lines[i]
        
        if(l.find("Nombre") != -1):
            Tabla0val.append(l[23:])
        
        if(l.find("Municipio") != -1):
            idxVereda = l.find("Vereda")
            idxVereda1 = idxVereda - 1
            idxVereda2 = idxVereda + 8
            
            Tabla0val.append(l[10:idxVereda1])
            Tabla0val.append(l[idxVereda2:])
        if(l.find("Finca") != -1):
            idxVereda = l.find("Código")
            idxVereda1 = idxVereda - 1
            idxVereda2 = idxVereda + 14
            
            Tabla0val.append(l[7:idxVereda1])
            Tabla0val.append(l[idxVereda2:])
        
        if(l.find("Grupo") != -1):
            Tabla0val.append(l[7:])
        
        if(l.find("Fecha de recibo") != -1):
            Tabla0val.append(l[17:])
            
        if(l.find("Fecha de análisis") != -1):
            Tabla0val.append(l[20:])
    
        if(l.find("Número de orden") != -1):
            Tabla0val.append(l[17:])
        
    
    
    
    
    #################################################
    
    # Granulomémtrie
    
    Tabla1Item = []
    Tabla1val= []
    
    for i in range(0,8):
        idxM = idxMalla + 2 + i
        idxVal = idxMalla + 12 + i
        
        colname = "Malla " + lines[idxM]
        Tabla1Item.append(colname)
        Tabla1val.append(lines[idxVal])
        #print("Malla: {}  value: {}".format(lines[idxM], lines[idxVal]))
    
    #print len(Tabla1Item)
    #print len(Tabla1val)
    
    #################################################
    
    # Caractéristiques physiques - éspaces vides aléatoires
    Tabla3Item = []
    Tabla3val = []
    
    for i in range(0,7):
        idxIt = idxItem + 4 + i
        Tabla3Item.append(lines[idxIt])
        #print("Item: {}".format(lines[idxIt]))
    
    idxItemFactor = lines.index("Factor de Rendimiento")
    
    s = ""
    for i in range(idxItemFactor,idxDefectos -1):
        # + 1 pour éviter le "factor de rendimiento"
        if str(lines[i+1]) == s:
            continue
        
        Tabla3val.append(lines[i+1])
        #print("Value: {}".format(lines[i+1]))
    
    # supprime le 1er (si c'est 8, le titre de la collone se met en première place)
    if len(Tabla3val) == 8:
        Tabla3val = Tabla3val[1:]
    
    #print len(Tabla3Item)
    #print len(Tabla3val)
    
    
    
    #################################################
    
    # Defectos
    
    Tabla2Item = []
    Tabla2val = []
    
    # index of the second (%) -> because right before the defectos values
    indices = [i for i, x in enumerate(lines) if x == "(%)"]
    indice = indices[1]
    
    for i in range(0, 20):
        idxIt = idxDefectos + 2 + i
        idxVal = indice + 1 + i
        
        Tabla2Item.append(lines[idxIt])
        Tabla2val.append(lines[idxVal])
        
    Tabla2val = ' '.join(Tabla2val).split()
    Tabla2Item = Tabla2Item[:-1]
    
    
    
    #print len(Tabla2Item)
    #print len(Tabla2val)
    
    #################################################
    
    # Caracteristicas
    
    Tabla4Item = []
    Tabla4val = []
    
    for i in range(0,10):
        idxIt = idxCarac + 4 + i
        idxVal = idxCarac + 17 + i
        
        Tabla4Item.append(lines[idxIt])
        Tabla4val.append(lines[idxVal])
    #print len(Tabla4Item)
    #print len(Tabla4val)
    
    #################################################
    
    # Description de la taza
    
    possibleValues = ["Limpia","Agrio","Vinagre","Fermento","Stinker","Metálico",
                      "Maderoso","Acre","Reposo","Terroso","Moho","Fenol",
                      "Químico","Quimico","Humo","Contaminado"]
    
    Tabla5Item = ["Taza 1","Taza 2","Taza 3","Taza 4","Taza 5"]
    Tabla5val = []
    
    found = False
    while found == False:
        idxTaza = idxTaza + 1
        #print("Line: {} : {}".format(idxTaza, lines[idxTaza]))
        if str(lines[idxTaza]) in possibleValues:
            found = True
    
    
    for i in range(0,5):
        idxVal = idxTaza  + i
        Tabla5val.append(lines[idxVal])

        
    items = Tabla0Item + Tabla1Item + Tabla2Item + Tabla3Item + Tabla4Item + Tabla5Item
    values = Tabla0val + Tabla1val + Tabla2val + Tabla3val + Tabla4val + Tabla5val
    
    valuesTaza.append(values)
    

df = pd.DataFrame.from_records(valuesTaza, columns=items)
df

,Nombre Caficultor,Municipio,Vereda,Finca,SICA,Grupo,Fecha Recpt,Fecha Analysis,No Orden,Malla 18,...,Dulzor,Uniformidad,Balance,Taza limpia,Puntaje catador,Taza 1,Taza 2,Taza 3,Taza 4,Taza 5
0,Jorge Iván Ospina Alzate,Marsella,Corozal,La Renata,06644001781,Lote 6.1,2011-01-25,2011-01-25,5477,42.38,...,0.0,0,0.0,0,0.0,Fermento,Fermento,Fermento,Fermento,Fermento
1,Jorge Iván Ospina Alzate,Marsella,Corozal,La Renata,06644001781,Lote 8,2011-01-25,2011-01-25,5478,20.18,...,0.0,0,0.0,0,0.0,Quimico,Quimico,Quimico,Quimico,Quimico
2,Jose Climaco Diaz,Balboa,Tambores,El Bosque,06607500516,TERCER CONCURSO DEPARTAMENTAL 2011,2011-12-02,2011-12-05,5854,35.52,...,10.0,10,7.0,10,7.0,Limpia,Limpia,Limpia,Limpia,Limpia
3,Gladys Noreña Tabares,Balboa,La Floresta,La Mejora,06607501187,TERCER CONCURSO DEPARTAMENTAL 2011,2011-12-02,2011-12-05,5855,40.05,...,10.0,10,7.0,10,7.0,Limpia,Limpia,Limpia,Limpia,Limpia
4,Jairo Antonio Noreña,Balboa,La Floresta,La Nubia,06607500304,TERCER CONCURSO DEPARTAMENTAL 2011,2011-12-02,2011-12-05,5856,37.62,...,10.0,10,7.0,10,7.0,Limpia,Limpia,Limpia,Limpia,Limpia
5,Jorge Marin Velez,Balboa,Carminales,La Estrella,06607500533,TERCER CONCURSO DEPARTAMENTAL 2011,2011-12-06,2011-12-07,5857,42.68,...,10.0,10,6.5,10,6.75,Limpia,Limpia,Limpia,Limpia,Limpia
6,Luis Alberto Cardona,Balboa,San Antonio,El Reposo,06607501128,TERCER CONCURSO DEPARTAMENTAL 2011,2011-12-06,2011-12-07,5858,44.92,...,10.0,10,7.25,10,7.5,Limpia,Limpia,Limpia,Limpia,Limpia
7,Jorge Ivan Cardona,Balboa,San Antonio,El Reposo,06607500809,TERCER CONCURSO DEPARTAMENTAL 2011,2011-12-06,2011-12-07,5859,39.5,...,10.0,10,7.0,10,7.5,Limpia,Limpia,Limpia,Limpia,Limpia
8,Miller Cuellar Vargas,La Celia,Montblanc,El Truco,06638301425,TERCER CONCURSO DEPARTAMENTAL 2011,2011-12-06,2011-12-07,5860,31.07,...,10.0,10,6.5,10,6.5,Limpia,Limpia,Limpia,Limpia,Limpia
9,Otoniel Moreno Aguirre,Balboa,La Cienaga,El Lotecito,06607500288,TERCER CONCURSO DEPARTAMENTAL 2011,2011-12-06,2011-12-07,5861,51.4,...,10.0,10,6.25,10,6.5,Limpia,Limpia,Limpia,Limpia,Limpia


In [48]:
with pd.option_context('display.max_rows', None, 'display.max_columns', 49):
    print(df)

            Nombre Caficultor         Municipio            Vereda  \
0    Jorge Iván Ospina Alzate          Marsella           Corozal   
1    Jorge Iván Ospina Alzate          Marsella           Corozal   
2           Jose Climaco Diaz            Balboa          Tambores   
3       Gladys Noreña Tabares            Balboa       La Floresta   
4        Jairo Antonio Noreña            Balboa       La Floresta   
5           Jorge Marin Velez            Balboa        Carminales   
6        Luis Alberto Cardona            Balboa       San Antonio   
7          Jorge Ivan Cardona            Balboa       San Antonio   
8       Miller Cuellar Vargas          La Celia         Montblanc   
9      Otoniel Moreno Aguirre            Balboa        La Cienaga   
10    Jairo Antonio Cano Cano            Balboa        El Chuzcal   
11          Luis Enrique Meza   Belen De Umbria            Andica   
12    William Osorio Espinosa   Belen De Umbria       El Aguacate   
13    Oswaldo Hurtado Oquendo   Be

In [47]:
df.to_csv("TazaDatos.csv", sep=',', encoding='utf-8')